# What's this
- 主にチーム内での情報共有を目的とした文書ですがせっかくなのでpublicにしています
- このNotebookは本コンペのルールの翻訳など、とにかく参加する前に読んでおくと良さそうなことをまとめていきます


# ルール
- プレイヤー数 4人
- プレイヤーは5000Haliteと1つのShipを持ってゲームを開始する
- ShipをShipyardに変換するには500Haliteが必要
- Shipyardから Shipを作るには500Haliteが必要
- ターンごとに制御している各船に対して東西南北の異動またはShipyardへの変換コマンドが可能
- 船にコマンドが発行されない場合、船は移動せず、代わりに現在の位置の下にあるHaliteを25%採掘する
- 採掘されたHaliteはShipに保持される
- ShipのHalite保持量に上限はなし
- ShipのHaliteはShipyardに収めるまで購入などに利用できない
- プレイヤーはShipをShipyardに移動してHaliteを積み下ろし、HomeworldにteleportさせることでHaliteの最終スコアに加算することができる
  - 訳者メモ)teleportのくだりは自動で行われるのでShiptyardに積み下ろせば資産になると思っておけばいい
- 複数のShipが同じマスに入ると衝突が起きる
- このとき最も小さいShip(最も保持するHaliteの少ない)のみが生き残りほかはすべて破壊されます
- 生き残ったShipは破壊される他のすべてのShipのHaliteを盗み取ります
- Haliteを保持量が引き分けの場合全てのShipが破壊されます
- Shipの衝突判定後Shipyardとの衝突判定が行われ、敵のShipがShipyardに衝突した場合、その両方が破壊されます
- プレイヤーはすべてのShipとShipyardを失うとゲームから除外される
- マップのHaliteはゲーム開始時にランダムに(ただし対称に)配置される
- 各マスのHaliteは1ターンあたり2%、最大500まで回復する
  - 訳者メモ)既存のHaliteに比例するので0になると回復しなくなる
- ゲームは最大400ターン

# 勝利条件
- 400ターン終了までに最も多くのHaliteを集める
- またはそれまでに全てのほかプレイヤーを全滅させる 

# Game Setup
- ゲームボードは21x21セルの大きさで、北/南と東/西の境界の両方を囲んでいます。
- ボードの南西（左下）コーナーは（0,0）と指定され、ボードの北東（右上)コーナーは（20,20）と指定されます。
- 開始プレイヤーの位置は、（5,5）、（15,5）、（5,15）、および（15,15）です。- - ゲームコードでは、位置はシリアル化されたリストとして与えられます。各セルは、[position = row * 21 + column].という式で計算される整数です。


- マップは岩塩のランダムな分布で覆われています。
- この分布は、垂直方向と水平方向の両方で対称です。
- 各セルには、500個以下の岩塩があります（セルが持つことができる最大量）。
- 合計でゲームボードは24,000の岩塩から始まります。

- 戦略メモ:多くのプレイヤーは第1ターンにShipyardを作成し多くのSihpを展開できるように行動するでしょう


# TurnOrder
- Haliteの各ゲームは400ターン、または1人を除くすべてのプレイヤーが排除されるまで続きます。
- 各ターンには、各エージェントにボードステートのコピー（「observation」）が与えられ、ゲームのすべての側面に関する完全な情報（各プレーヤーが持っているHaliteの量、すべての船の位置（および彼らが持っているHaliteの量）が含まれます そして、ボードのすべてのセルにどれくらいの量のHaliteがあるか。
- 各エージェントは、設定された時間内に彼らの船と造船所が取るための行動のリストを返す必要があります。
- エージェントが時間内にアクションを返さない場合、またはエラーが発生した場合は、エラーとしてマークされ、すべての船と造船所がすぐにゲームから削除されます。 
- これにより、プレーヤーは最終スコア順で最下位に配置されます（すでに脱落したプレーヤーよりも）。
  - 訳者メモ)時間切れのリスクが非常に高いのでとにかく行動を返せるようにすること。
  
  
# Command
## Move
Actions: NORTH, SOUTH, EAST, WEST
Cost: 0 halite

- このアクションにより、船は要求された方向に1スペース移動します。
- 複数の船が同じスペースでターンを終了すると、衝突が発生します
- 船が友好的な造船所に移動すると、運んでいたHaliteが即座に積み下ろしされ、プレイヤーの総Haliteに追加されます。
- Haliteの積み下ろしは衝突後に解決されることに注意してください。

- Halite IIIとは異なり、このゲームでは船の移動にhaliteは必要ありません。

## Hold / Mine
Actions: N/A
Cost: 0 halite


- 船にターンの命令が出されない場合、船は移動せずに現在の位置を保持します。
- また、可能であれば、そのセルから岩塩を採掘します。
- 船は、セルの上部にあるターンごとに、セルの総岩塩の25％を採掘します。
- 採掘された岩塩はその個々の船の貯蔵庫に追加され、ゲーム終了時のスコアにカウントするには造船所に積み下ろす必要があります。
- 衝突はマイニングの前に解決されることに注意してください。

## Convert
Actions: CONVERT
Cost: 500 halite


- 船が「Convert」アクションを発行した場合、船は自分がいるスペースに新しい造船所を作成するためにそれ自体を破壊します。
- ターン解決中の衝突の前に変換が行われます。
- 新しい造船所の下にあるセルのHaliteは破壊されます。
- Convertした船によって運ばれていた岩塩は即座に積み下ろしします。
- 船をConvertするための500の岩塩コストは、まず船が保持している量から差し引かれ、残っているコストはプレイヤーの岩塩の合計から削除されます。
- つまり、総岩塩と船に搭載された岩塩の合計が500岩塩よりも大きい場合は、岩塩が500未満でも船を転用できます。
- 自分の岩塩と船の岩塩の間に500未満の岩塩がある場合、このアクションは失敗し、「Hold」コマンドとして解決されます。
- Convertしようとしている船が現在造船所の上にある場合も、Convertアクションは失敗します。

- 戦略のヒント：衝突の前にConvertが行われるため、敵に追われていて、たくさんHaliteを保持しているShipからHaliteを安全に回収する方法としてConvertを選択できます

## Spawn Ship
Actions: SPAWN
Cost: 500 halite


- 造船所がSPAWNコマンドを発行した場合、造船所はその上に新しい船をスポーンしようとします。 
- SPAWNコマンドは、それを発行したプレイヤーが注文を完了するのに十分なHaliteを持っている場合にのみ成功します。
- Spawnは衝突の前に行われるため、新しい船は敵（または味方の船）と衝突し、作成されたターンに破壊される可能性があります。

- 戦略のヒント：他の船が造船所に岩塩を積み下ろしさせようとしているときは、Spawnしないようにすることが重要です。敵の船があなたの造船所を攻撃しようとしている場合は、船をスポーンして彼らの船と衝突させることで防衛できることを覚えておいてください。

## Hold
Actions: N/A
Cost: 0 halite

- 造船所にコマンドが送信されない場合、ターンには何もしません。


# Turn Resolution
- 各ターンは以下のフェーズで解決される

1. Spawning
- スポーンアクションを注文したすべての造船所は、スポーンした造船所の上のボードに新しい船が追加されることで解決されます。このフェーズでは、複数の船/造船所がボード上の同じスペースを占めることができます-これらの重複は衝突解決で解決されます。スポーンはボードポジション順（左上が最初）で解決されます。これは、プレイヤーがすべてのスポーンオーダーを解決して途中で岩塩を使い果たした場合に関連します。
  - 訳者メモ)作りすぎた場合右下の方には生成されない
2. Conversion
- 造船所に改造しようとしたすべての船が解決されます。十分な資金があり、既存の造船所の上にない場合、船は造船所になります。造船所の変換はボードポジション順（左上が最初）で解決されます。これは、プレーヤーがすべての変換順序を解決して途中で岩塩を使い果たした場合に関連します。
3. Movement
- すべての船は彼らの命令に従って動かされます。船は衝突することなく、互いに「通過」することができます（それぞれが前のスペースに移動する場合）。
  - 訳者メモ)すれ違える模様
4. Ship Collision
- 船の衝突が解決され、ボード上のすべてのセル内の船の数が1つ以下に減少します。衝突するたびに、最小の船が生存者となり、他のすべてが破壊されます。最小の船とは、保管場所での岩塩が最も少ない船と定義されます。複数の船が最小の塩分を持っているためにタイである場合、衝突のすべての船は破壊されます。衝突では、どのプレイヤーがどの船を所有しているかは考慮されないことに注意してください。友好的な船同士の衝突が発生する可能性があります。生き残った船は破壊された船の貨物からすべての岩塩を取り、ボード上のその場所を保持します。
5. Shipyard Collision
- 異なるプレイヤーの船と造船所がボード上の同じセルを占めている場合、両方が破壊されます。船が持っていた岩塩は失われます。造船所を破壊しても総Haliteは減りませんが、堆積したすべての塩酸塩はすでに安全にテレポートされて故郷に戻っています。
6. Halite Depositing
- 友好的な造船所の上にあるすべての船は、個々の倉庫に持っているすべての岩塩を預けます。
7. Halite Mining
- 岩塩の上に位置を保持していたすべての船が採掘をし、セルの岩塩の25％をその船の倉庫に移動します。船の保管中の岩塩は、船の上部にある青い輝きでビジュアライザに表示されます。
8. Halite Regeneration
- 0個を超える岩塩があり、その上に船がないボード上のすべてのセルは、セル内の岩塩の既存の量の2％だけ再生します。岩塩はセルあたり最大500岩塩まで成長できます。
9. End Turn
- 「Step」番号が増加し、ターンが終了します。更新されたボードは保存され、「observation」として各エージェントに再配布され、次のターンに移動します。

- このステップで、プレイヤーがゲームで生き残ることができなくなった場合、そのプレイヤーは排除されます。これは、プレイヤーがもはや船を持たず、500未満の岩塩が残っているか、残りの造船所がない場合に発生します（したがって、これ以上船をスポーンできません）。除去された場合、プレーヤーの残りの造船所はすべてゲームから削除されます。

- 各ターンの終わりに、ゲームは1人のプレーヤーを除いてすべてが排除されたか、ゲームが400ターンに達したかどうかもチェックします。新しいターンを開始する代わりにどちらかが当てはまる場合、ゲームはゲーム終了ステップに移動します。

# Game End
- ゲームが終了すると、プレーヤーは最終的に収集された岩塩に基づいてランク付けされます。 船、造船所、未堆積の岩塩はすべて、ゲームの終わりには何の価値もありません。 プレーヤーは、ゲームを終了した総Haliteの順でランク付けされます。 収集された総岩塩で同着のプレーヤーは、ゲームのランキングでも同着です。 除去されたプレーヤーは、収集された総ハライトのスコアが0になり（除去されたときのハライトの量に関係なく）、ゲームから除去された順序で互いにランク付けされます。 最後に、エラーが発生したすべてのボットがランキングの最後に結合されます。

- 最終的なランキングは評価システムに送られ、各プレーヤーのスキルレーティングが変更されます。 詳細については、評価タブをご覧ください。

# Evaluation
- チームは毎日最大5つのエージェント（ボット）をコンテストに提出できます。各提出物は、同様のスキルレーティングを持つラダー上の他のボットに対してエピソード（ゲーム）をプレイします。時間の経過とともに、スキルレーティングは勝利とともに上昇し、損失とともに低下します。提出されたすべてのボットは、コンテストが終了するまでゲームをプレイし続けます。リーダーボードには、最高のスコアリングボットのみが表示されますが、[送信]ページですべての送信の進行状況を追跡できます。

- 各提出には、ガウスN（μ、σ2）によってモデル化された推定スキルレーティングがあります。ここで、μは推定スキルであり、σは時間の経過とともに減少するその推定の不確実性を表します。

- 提出物をアップロードすると、最初に検証エピソードが再生され、その提出物が自身のコピーと対戦して、正常に動作することを確認します。エピソードが失敗した場合、提出物はエラーとしてマークされます。それ以外の場合は、提出をμ0= 600で初期化し、進行中の評価のためにすべての提出のプールに参加します。

- 私たちはすべての提出物のプールからエピソードを繰り返し実行し、公正な一致のために同様の評価を持つ提出物を選択しようとします。私たちは、提出ごとに1日あたり最大8エピソードを実行することを目指しています。最新の提出エピソードでは、フィードバックをより速く提供できるように、レートをさらに少し上げています。

- エピソードが終了すると、そのエピソードのすべての提出物の評価の見積もりが更新されます。 1つの送信が勝った場合、そのμを増やし、その対戦相手のμを減らします-結果が引き分けである場合、2つのμ値を平均に近づけます。更新は、以前のμ値に基づいて期待される結果からの偏差に関連する大きさを持ち、各提出物の不確実性σにも関連しています。また、結果によって得られる情報量に関連してσ項を減らします。ボットがエピソードを獲得または喪失するスコアは、スキルレーティングの更新に影響しません。

- Haliteは4人用ゲームなので、解像度はもう少し複雑です。 4つのエージェントのエピソードを6つの2つのエージェントのエピソード（エージェント1対エージェント2、エージェント1対エージェント3、エージェント2対エージェント3など）として扱い、各ペアの標準的な2プレーヤースコアの更新を計算して、平均します。これらの更新は、各エージェントの最終的なランキングの更新を計算します。

- 提出期限に、追加の提出はロックされます。ゲームを実行し続けるために、さらに1週間が割り当てられます。この最後の週の終わりに、リーダーボード確定します。

- submitまでの流れについて
https://www.kaggle.com/iicyan/japanese-halite-submit

- SDKについて
https://www.kaggle.com/iicyan/japanese-halite-sdk-overview